In [116]:
%%writefile "/public/home/lizw/task/pore_c/website/generate_porec_matrix.py"
import pandas as pd
df_merge_part = pd.read_parquet('/public/home/lizw/task/pore_c/porec_1000_filter_mainchr_result/porecplot_v3/porec_pre_v2.9/DpnII_run08.order_2_merge_part.parquet')
#keep all for website matrix
anchor_region = pd.read_table('/public/home/lizw/data/IMPORTANT_genome/arabidopsis/tair10/all_chr.bed',header=None,names=['chrom','start','end'],converters={'chrom':str, 'start':int, 'end':int},comment="#")

binsize=500000
pd.options.mode.chained_assignment = None
df_region_filter_merge = pd.DataFrame([])
chrinterval_all = pd.DataFrame([])
df_bins_dict = {}
column = 'pos'
df = df_merge_part
for i in anchor_region.itertuples(index=False):
    region_chr,region_start,region_end = i
    query_text = f'(chrom == @region_chr) and ({column} > @region_start) and ({column} <=  @region_end)'
    df_region_filter_temp = df.query(query_text)
    df_region_filter_temp['region_start'] = region_start
    df_region_filter_temp['region_end'] = region_end
    df_cut,df_bins= pd.cut(df_region_filter_temp[column],bins=range(region_start,region_end+binsize-1,binsize),precision=0,retbins=True)
    df_region_filter_temp['interval']  = df_cut
    chrinterval_all = df_region_filter_merge.append(df_region_filter_temp)
    chrinterval_tmp = pd.DataFrame(pd.cut(np.arange(region_start+binsize,region_end+binsize,binsize),bins=df_bins,precision=0),columns = ['interval'])
    chrinterval_tmp['chrom'] = region_chr
    chrinterval_all = chrinterval_all.append(chrinterval_tmp)

df_merge_allinterval = pd.merge(df_region_filter_merge,chrinterval_all,on=['chrom','interval'],how='outer')

df_merge_allinterval.sort_values(by=['chrom','interval'],inplace=True)
interval_df = df_merge_allinterval['interval'].drop_duplicates().reset_index(drop=True)
interval_df = pd.DataFrame(interval_df)
interval_df_dict = {y:x for x,y in interval_df.itertuples()}
df_merge_allinterval['for_group'] = df_merge_allinterval['interval']
df_merge_allinterval['for_group'].replace(interval_df_dict,inplace=True)
df_merge_allinterval['chr_interval'] = df_merge_allinterval['for_group'].astype('str').str.cat(df_merge_allinterval['chrom'].astype('str'),sep="_")
heat = df_region_filter_merge.groupby(by=['read_name','chr_interval'],sort=False).count()['pos'].astype(np.int16).unstack().fillna(0)

heat.to_parquet("/public/home/lizw/task/pore_c/website/porec_matrix_all.parquet")
interval_df.to_csv("/public/home/lizw/task/pore_c/website/interval_id")

Writing /public/home/lizw/task/pore_c/website/generate_porec_matrix.py
